In [2]:
from datasets import load_dataset
dataset = load_dataset("go_emotions")

dataset

Generating test split: 100%|█████████████████████████████████████████████████████████| 5427/5427 [00:00<00:00, 248993.50 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 43410
    })
    validation: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5426
    })
    test: Dataset({
        features: ['text', 'labels', 'id'],
        num_rows: 5427
    })
})